In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
import requests

import warnings
warnings.filterwarnings('ignore')

import os
import selenium
from selenium import webdriver
import re
from datetime import datetime as dt, timedelta
import datetime
from tqdm import tqdm

import pandas as pd
import time
import math
import numpy as np

from dateutil.parser import parse
from selenium.webdriver import Chrome, ChromeOptions
import lxml
import lxml.html

In [5]:
b = pd.read_excel('./comp_info.xlsx').iloc[:,1:]
b

,category,name,homepage,upjong,geo,foundation,employee,sal_min,sal_mid,sal_max,bonus_min,bonus_mid,bonus_max,average,career,wlb,money,culture,board
0,서비스업,스타벅스커피코리아,www.istarbucks.co.kr,외식/프랜차이즈,서울특별시 중구,1997,"10,000+(명)","2,100만원","3,326만","8,100만원",30만원,400만,"2,000만원",2.8,2.8,2.6,2.5,2.8,2.1
1,서비스업,한국맥도날드,www.mcdonalds.co.kr,외식/프랜차이즈,서울특별시 종로구,1986,"1,001~5,000(명)","2,800만원","3,656만",1.3억원,20만원,300만,"1,500만원",3.3,3.2,2.9,2.7,3.3,3.0
2,서비스업,하나투어,www.hanatourcompany.com,여행사/여행서비스업,서울특별시 종로구,1993,"1,001~5,000(명)","2,304만원","4,495만",1.0억원,30만원,200만,"3,000만원",2.8,2.3,4.0,2.0,3.1,1.8
3,서비스업,유베이스,www.ubase.co.kr,사업지원/시설관리/임대,서울특별시 용산구,1999,"10,000+(명)","2,191만원","3,789만","7,374만원",80만원,300만,540만원,2.3,2.0,2.9,1.9,2.3,1.9
4,서비스업,트랜스코스모스코리아,www.trans-cosmos.co.kr,고용알선/인력공급업,서울특별시 용산구,2001,"5,001~10,000(명)","2,219만원","3,870만","7,312만원",50만원,120만,200만원,2.8,2.5,3.1,2.1,2.7,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,기관/협회,국토연구원,krihs.re.kr,협회/단체,세종시,1978,51~200(명),"2,457만원","4,779만","9,752만원",60만원,250만,400만원,3.5,3.6,4.2,2.8,3.2,3.5
704,기관/협회,한국식품연구원,kfri.re.kr,연구개발업,경기도 성남시 분당구,2007,201~500(명),"2,160만원","4,800만",1.0억원,100만원,500만,"2,000만원",3.4,3.2,4.4,3.1,3.0,2.5
705,기관/협회,법무법인세종,www.shinkim.com,법무 서비스업,서울특별시 중구,1997,"501~1,000(명)","3,000만원","6,360만",4.8억원,30만원,"1,000만",2.0억원,3.0,2.8,3.1,3.1,2.8,2.2
706,기관/협회,세종문화회관,www.sejongpac.or.kr,국가/공공기관,서울특별시 종로구,1999,201~500(명),"2,061만원","3,900만","7,296만원",NaN,NaN,NaN,3.1,2.8,3.7,2.9,2.7,2.3


In [2]:
a = pd.read_excel('./comp_review.xlsx').iloc[:,1:]
a

FileNotFoundError: [Errno 2] No such file or directory: './comp_review.xlsx'

In [4]:
len(np.unique(a.name))

708

In [4]:
comp_list = pd.read_excel('./data/comp_list.xlsx').iloc[:,1:]
comp_list

,서비스업,제조/화학,의료/제약/복지,유통/무역/운송,교육업,건설업,IT/웹/통신,미디어/디자인,은행/금융업,기관/협회
0,스타벅스커피코리아,삼성전자,한미약품,씨제이올리브영,해커스 교육그룹,한국토지주택공사,쿠팡,씨제이씨지브이,국민건강보험공단,대한무역투자진흥공사
1,한국맥도날드,엘지전자,대웅제약,에프알엘코리아,동화세상에듀코,한전KPS,㈜티몬,롯데시네마,새마을금고,삼일회계법인
2,롯데리아,엘지디스플레이,종근당,한샘,에스티유니타스,현대엔지니어링,위메프,씨제이이앤엠,근로복지공단,한영회계법인
3,하나투어,현대자동차,녹십자,이마트,에듀윌,현대건설,삼성에스디에스,교원,농협은행,안진회계법인
4,유베이스,엘지이노텍,한국유나이티드제약,홈플러스,대교,대우건설,엘지씨엔에스,문화방송,중소기업은행,삼정회계법인
...,...,...,...,...,...,...,...,...,...,...
95,마이크임팩트,메디앙스,펜믹스,올림푸스한국,창의와탐구,평화엔지니어링,데이터스트림즈,아이코닉스,예금보험공사,서울산업진흥원
96,부킹닷컴코리아,한국가스공사,종근당바이오,필립스코리아,하이컨시,비엔에이치,위메이드,엠디마케팅,카카오페이,한국식품연구원
97,한국기업데이터,타이코에이엠피,경희의료원,트라이씨클,글로벌콘텐츠리퍼블릭,세아네트웍스,이카운트,한국학술정보,제이비우리캐피탈,법무법인세종
98,바이오톡스텍,에스케이케미칼,세이브더칠드런코리아,브리티쉬아메리칸토바코코리아,제이엠커리어,경남기업,에스케이브로드밴드,한국우편사업진흥원,엔에이치농협캐피탈,세종문화회관


In [22]:
comp_review = pd.DataFrame(columns=['category','name','score','title','pros','cons'])

In [6]:
comp_list_pre = pd.DataFrame(columns=['category','name'])
for i in comp_list.columns:
    t = pd.DataFrame({'category':i, 'name':comp_list[i]})
    comp_list_pre = pd.concat([comp_list_pre, t], axis=0)
    
comp_list_pre = comp_list_pre.reset_index(drop=True)

In [7]:
comp_list_pre

,category,name
0,서비스업,스타벅스커피코리아
1,서비스업,한국맥도날드
2,서비스업,롯데리아
3,서비스업,하나투어
4,서비스업,유베이스
...,...,...
995,기관/협회,서울산업진흥원
996,기관/협회,한국식품연구원
997,기관/협회,법무법인세종
998,기관/협회,세종문화회관


In [7]:
new_comp_list = pd.read_excel('comp_list_not.xlsx').iloc[:,1:]
new_comp_list

,name,new_comp_name
0,롯데리아,롯데지알에스
1,씨제이푸드빌,CJ푸드빌
2,에스케이쉴더스,NaN
3,빕스,NaN
4,던킨/던킨도너츠,NaN
...,...,...
287,서울시교육청,서울특별시교육청
288,인포마스터,NaN
289,사회복지법인사회복지공동모금회,NaN
290,언스트앤영어드바이저리,NaN


In [8]:
new_comp_list2 = new_comp_list[~new_comp_list['new_comp_name'].isnull()]
new_comp_list2

,name,new_comp_name
0,롯데리아,롯데지알에스
1,씨제이푸드빌,CJ푸드빌
7,에이디티캡스,SK쉴더스
8,세일즈웍스 코리아,세일즈웍스코리아
10,그랜드코리아레저,GKL
...,...,...
279,한국직업능력개발원,한국직업능력연구원
282,법무법인광장,법무법인 광장
286,한국환경정책평가연구원,한국환경연구원
287,서울시교육청,서울특별시교육청


In [9]:
new_comp_list2 = pd.merge(new_comp_list2, comp_list_pre)
new_comp_list2

,name,new_comp_name,category
0,롯데리아,롯데지알에스,서비스업
1,씨제이푸드빌,CJ푸드빌,서비스업
2,에이디티캡스,SK쉴더스,서비스업
3,세일즈웍스 코리아,세일즈웍스코리아,서비스업
4,그랜드코리아레저,GKL,서비스업
...,...,...,...
117,한국직업능력개발원,한국직업능력연구원,기관/협회
118,법무법인광장,법무법인 광장,기관/협회
119,한국환경정책평가연구원,한국환경연구원,기관/협회
120,서울시교육청,서울특별시교육청,기관/협회


In [2]:
new_comp_list2 = pd.read_excel('blind_comp_info.xlsx').iloc[:,1:]
new_comp_list2

,category,name,homepage,upjong,geo,foundation,employee,sal_min,sal_mid,sal_max,bonus_min,bonus_mid,bonus_max,average,career,wlb,money,culture,board,new_comp_name
0,서비스업,스타벅스커피코리아,www.istarbucks.co.kr,외식/프랜차이즈,서울특별시 중구,1997,"10,000+(명)","2,100만원","3,326만","8,100만원",30만원,400만,"2,000만원",2.8,2.8,2.6,2.5,2.8,2.1,NaN
1,서비스업,한국맥도날드,www.mcdonalds.co.kr,외식/프랜차이즈,서울특별시 종로구,1986,"1,001~5,000(명)","2,800만원","3,656만",1.3억원,20만원,300만,"1,500만원",3.3,3.2,2.9,2.7,3.3,3.0,NaN
2,서비스업,하나투어,www.hanatourcompany.com,여행사/여행서비스업,서울특별시 종로구,1993,"1,001~5,000(명)","2,304만원","4,495만",1.0억원,30만원,200만,"3,000만원",2.8,2.3,4.0,2.0,3.1,1.8,NaN
3,서비스업,유베이스,www.ubase.co.kr,사업지원/시설관리/임대,서울특별시 용산구,1999,"10,000+(명)","2,191만원","3,789만","7,374만원",80만원,300만,540만원,2.3,2.0,2.9,1.9,2.3,1.9,NaN
4,서비스업,트랜스코스모스코리아,www.trans-cosmos.co.kr,고용알선/인력공급업,서울특별시 용산구,2001,"5,001~10,000(명)","2,219만원","3,870만","7,312만원",50만원,120만,200만원,2.8,2.5,3.1,2.1,2.7,2.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,미디어/디자인,에이치에스애드,www.hsad.co.kr,광고업,서울특별시 마포구,2004,"501~1,000(명)","2,454만원","5,000만",1.6억원,100만원,700만,"4,000만원",3.5,3.5,3.1,3.2,3.2,2.6,HS%20Ad
826,미디어/디자인,클콩,superbean.tv,컨텐츠/엔터테인먼트,경기도 성남시,2012,51~200(명),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%ED%81%B4%EC%BD%A9
827,미디어/디자인,현대퓨처넷,hyundaifuturenet.co.kr,IT/소프트웨어,서울특별시 서초구,1992,1~50(명),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%ED%98%84%EB%8C%80%ED%93%A8%EC%B2%98%EB%84%B7
828,은행/금융업,산와대부,sanwamoney.co.kr,금융업,서울특별시 관악구,2002,1~50(명),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%EC%82%B0%EC%99%80%EB%8C%80%EB%B6%80


In [3]:
new_comp_list2 = new_comp_list2[~new_comp_list2['new_comp_name'].isnull()].reset_index(drop=True)[['category','name','new_comp_name']]
new_comp_list2

,category,name,new_comp_name
0,서비스업,롯데리아,롯데지알에스
1,서비스업,씨제이푸드빌,CJ푸드빌
2,서비스업,에이디티캡스,SK쉴더스
3,서비스업,세일즈웍스 코리아,세일즈웍스코리아
4,서비스업,그랜드코리아레저,GKL
...,...,...,...
117,미디어/디자인,에이치에스애드,HS%20Ad
118,미디어/디자인,클콩,%ED%81%B4%EC%BD%A9
119,미디어/디자인,현대퓨처넷,%ED%98%84%EB%8C%80%ED%93%A8%EC%B2%98%EB%84%B7
120,은행/금융업,산와대부,%EC%82%B0%EC%99%80%EB%8C%80%EB%B6%80


In [19]:
new_comp_list2[:1]

,category,name,new_comp_name
0,서비스업,롯데리아,롯데지알에스


In [23]:
op = ChromeOptions()
# op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')

browser = Chrome('./chromedriver.exe', options=op)
browser.get('https://www.teamblind.com/kr/')

login = browser.find_element_by_xpath('//*[@id="wrap"]/header/div/div/div/a[2]')
login.click()

time.sleep(15)

not_list = []
for i in tqdm(range(len(new_comp_list2))):
    try:
        cate = new_comp_list2.loc[i, 'category']
        name = new_comp_list2.loc[i, 'new_comp_name']
        url = 'https://www.teamblind.com/kr/company/' + name + '/reviews?page=1'
        browser.get(url)
        time.sleep(1)
        html = browser.page_source
        root = lxml.html.fromstring(html)
        root.cssselect(f'#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(2) > div.review_all > div:nth-child(1) > div > div.review_item_inr > div.rating > strong')[0].text_content().replace('Rating Score','')
        
        for page in range(1,6):
        
            url = 'https://www.teamblind.com/kr/company/' + name + f'/reviews?page={page}'
            browser.get(url)
            time.sleep(1)
            html = browser.page_source
            root = lxml.html.fromstring(html)

            try:
                for p in range(1, 21):
                    try:
                        review_score = root.cssselect(f'#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(2) > div.review_all > div:nth-child({p}) > div > div.review_item_inr > div.rating > strong')[0].text_content().replace('Rating Score','')
                        review_title = root.cssselect(f'#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(2) > div.review_all > div:nth-child({p}) > div > div.review_item_inr > h3 > a')[0].text_content().replace('“','').replace('”','')
                        review_pros = root.cssselect(f'#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(2) > div.review_all > div:nth-child({p}) > div > div.review_item_inr > div:nth-child(4) > div.parag > p:nth-child(1) > span')[0].text_content()
                        review_cons = root.cssselect(f'#wrap > section > div > div > div.cpctw > div > div > div > div > section:nth-child(2) > div.review_all > div:nth-child({p}) > div > div.review_item_inr > div:nth-child(4) > div.parag > p:nth-child(2) > span')[0].text_content()

                        data_to_insert = {'category': cate, 'name': name, 'score': review_score, 'title': review_title, 
                                          'pros': review_pros, 'cons': review_cons}

                        comp_review = comp_review.append(data_to_insert, ignore_index=True)
                    except:
                        pass
            except:
                pass
        
    except:
    
        not_list.append(name)
        print(name)
        pass
    
    time.sleep(1)

 78%|███████████████████████████████████████████████████████████████                  | 95/122 [46:33<11:44, 26.09s/it]

%ED%94%84%EB%A1%9C%EC%97%90%EC%8A%A4%EC%BD%A4


 80%|████████████████████████████████████████████████████████████████▍                | 97/122 [47:08<09:28, 22.75s/it]

%EB%91%90%EC%82%B0%EC%9D%B8%ED%94%84%EB%9D%BC%EC%BD%94%EC%96%B4


 82%|█████████████████████████████████████████████████████████████████▌              | 100/122 [48:06<08:09, 22.25s/it]

...

 98%|██████████████████████████████████████████████████████████████████████████████  | 119/122 [54:00<01:04, 21.65s/it]

%ED%98%84%EB%8C%80%ED%93%A8%EC%B2%98%EB%84%B7


 98%|██████████████████████████████████████████████████████████████████████████████▋ | 120/122 [54:03<00:32, 16.16s/it]

%EC%82%B0%EC%99%80%EB%8C%80%EB%B6%80


100%|████████████████████████████████████████████████████████████████████████████████| 122/122 [54:33<00:00, 26.84s/it]


In [24]:
comp_review

,category,name,score,title,pros,cons
0,서비스업,롯데지알에스,1.0,도망가세요,롯데라는 굴레 속에서 은행 대출은 잘 나오는다는점..?,어떤 상사를 만나느냐에 따라 생지옥과 개꿀이 결정됨.진짜 꼰대와 젊은 꼰대의 조함에...
1,서비스업,롯데지알에스,1.0,블라인드 평점을 믿게해준 회사,대기업이라는 이름 네임벨류그외는….? 아직 모르겠어서 못적겠음,출근길 난이도 최상 겪어본 사람만 안다연봉 낮음업무 많음휴가 쓰기 어려움칼퇴하는 부...
2,서비스업,롯데지알에스,1.0,일하는 기계가 무엇인지 체험할 수 있는 곳,월급을 제때 준다이사한 지 얼마 안돼서 사옥이 쾌적함,매번 급하게 들어오는 업무들로 인해 휴가 일정을 맞추거나 퇴근 후 개인 일정 잡기가...
3,서비스업,롯데지알에스,1.0,휴가가 많은데 쓰기가 어렵다,1. 회사커피머신에 엔제리너스 원두2. 회사 앞에 롯데가 붙음,1. 워라벨 최악2. 휴가 쓸 때 눈치줌3. 엘레베이터 3대 밖에 없어서 지각 확률 높음
4,서비스업,롯데지알에스,3.0,워라벨 굿,"식품업계 상위 복지 및 처우, 워라벨 보장이 장점이라 할 수 있음","사내정치, 경쟁력 약화중, 비전없음으로 전략개선이 시급하다"
...,...,...,...,...,...,...
9527,기관/협회,%EC%84%9C%EC%9A%B8%EC%82%B0%EC%97%85%EC%A7%84%...,2.0,"워라밸, 연봉 모두 불만족스러운 회사",정년 보장서울 근무ㅋ30자 채우기 힘드네.....,꼰들이 만들어내는 조직분위기소문이 빠른 조직신입한테 일몰아줌낮은 연봉야근수당 받으려...
9528,기관/협회,%EC%84%9C%EC%9A%B8%EC%82%B0%EC%97%85%EC%A7%84%...,3.0,그냥저냥,그냥저냥 다니기 좋음그럼에도 월급은 칼같이회사가 사라지지 않는한 인서울,그냥저냥 다녀도 안짤림보이지 않은 직원간 갈등개판 관리자 갑질 서울시
9529,기관/협회,%EC%84%9C%EC%9A%B8%EC%82%B0%EC%97%85%EC%A7%84%...,2.0,대충 다니는 놈이 편한 회사,고용 안정성월급 안밀림서울에 있음시키는 것만 하면 됨,불합리한 인사와 급여일하는 놈만 계속 시킨다대표이사가 좌지우지하는 꼴
9530,기관/협회,%EC%84%9C%EC%9A%B8%EC%82%B0%EC%97%85%EC%A7%84%...,1.0,모이면 아무도 모른척 하는 곳,혼자 많이 알면 그나마 버티기 괜찮은 곳좋은 사람들이 많은데 같이 일하면 안되는 곳,선배는 꼰대 후배는 무개념 팀장은 등산복 입고 쪼리에 반바지 입고 다니는 사람도 있...


In [25]:
comp_review.to_excel('./comp_review_not.xlsx')

In [29]:
pd.merge(comp_review, new_comp_list2[['name','new_comp_name']], how='left', left_on = 'name', right_on='new_comp_name').to_excel('./comp_review_not.xlsx')

In [ ]:

op = ChromeOptions()
op.add_argument('--headless')
op.add_argument('--no-sandbox')
op.add_argument('--disable-dev-shm-usage')

browser = Chrome('./chromedriver.exe', options=op)
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'

url = "https://news.naver.com/main/list.naver"

header = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "max-age=0",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
}

article_info = pd.DataFrame()

# 경제 부문만
category='경제'

#     # 시작 날짜부터 마지막 날짜까지
#     for date in range(start_date, end_date+1):

# 마지막 페이지 번호 구하기
for first_page in range(1,1000, 10):

    param = {
        "mode" : "LSD",
        "mid" : "sec",
        "sid1" : "101",
        "listType" : "summary", # 속보 중 "요약" 부문 가져오기
        "date" : f"{date}",
        "page" : f"{first_page}"
    }


    # 들어갈 링크 가져오기
    response = requests.get(url,params=param, headers=header).text
    soup = BeautifulSoup(response, 'html.parser')

    # "다음" 버튼이 있는지 없는지로 판단
    pre_btn = soup.find('a', class_='pre nclicks(fls.page)')
    next_btn = soup.find('a', class_="next nclicks(fls.page)")

    if next_btn != None:
        continue

    else:
        try:
            last_page = int(soup.find_all("a", class_="nclicks(fls.page)")[-1].text)
            print(f"{category} {date} 일자 최종 페이지 번호는 {last_page}입니다")                
            break

        except:
            try:
                paging_div = soup.find('div', class_='paging')
                last_page = int(paging_div.find_all('strong')[-1].text)
                print(f"{category} {date} 일자 최종 페이지 번호는 {last_page}입니다")
                break
            except:
                last_page = 200
                print('에러로 인한 last_page 임시 설정')
                break


    time.sleep(1)

# 마지막 페이지까지 기사별 링크 가져오기
all_news_links = []

for page in range(1, last_page+1): #last_page+1

    param = {
        "mode" : "LSD",
        "mid" : "sec",
        "sid1" : "101",
        "listType" : "summary",
        "date" : f"{date}",
        "page" : f"{page}"
    }

    # 아무런 기사가 없는 날에도 page는 1로 존재하는 경우에 대한 처리

    try:
        # 들어갈 링크 가져오기
        response = requests.get(url,params=param, headers=header).text
        soup = BeautifulSoup(response, 'html.parser')


        # 이미지가 있는 경우 하나의 기사당 모두 3개의 동일한 href를 가짐 - class명 등 구분 방법이 따로 없어 모두 가져온 후 set() 처리
        page_news_links = soup.find_all("a", class_="nclicks(fls.list)") # link명이 기사용과 다름


        for page_news_link in page_news_links:
            all_news_links.append(page_news_link.attrs['href'])

        time.sleep(1)

    except:
        break

all_news_links = set(all_news_links)

print("기사별 링크 크롤링 완료")

time.sleep(1)


# 실제 기사 속 내용 모두 가져오기

# article_info = pd.DataFrame()

for i, news_link in enumerate(tqdm(all_news_links)):

    try:
        # 기사 속 내용 가져오기
        time.sleep(0.5)
        article_response = requests.get(news_link, headers=header).text
        article_soup = BeautifulSoup(article_response, 'html.parser')

        browser.get(news_link)
        time.sleep(0.5)

        html = browser.page_source
        root = lxml.html.fromstring(html)

    except:
        pass


    # 잘못된 url이 있는 경우가 존재했음 - pass로 제외하고 진행
    try:               

        news_title = article_soup.find('h3', id='articleTitle').text
        news_title = re.sub(r"\[.*\]|\s-\s.*", "", news_title)

        news_date = article_soup.find('span', class_='t11').text

        url_detail = article_soup.find('a', class_='btn_artialoriginal').attrs['href']
        contents = article_soup.find('div', class_="_article_body_contents").text
        # article = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\n\t]", '', article_soup.find('div', class_="_article_body_contents").text)
        # article = re.sub(' +', ' ', article).lstrip()
        contents = contents.replace('// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}',"")

    # 좋아요 / 화나요 크롤링 코드 추가
        good_body = root.cssselect('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
        good = good_body[0].text_content()
        bad_body = root.cssselect('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count')
        bad = bad_body[0].text_content()
        source_body = root.cssselect('#articleBody > div.copyright > div > p')
        source = str(source_body[0].text_content().split('ⓒ ')[1]).split('.')[0]


        article_info = article_info.append(pd.DataFrame([[category, news_title, news_date, url_detail, contents, good, bad, source]],
                                                        columns=['CATEGORY','TITLE','NEWS_DATE','URL_DETAIL','clean_content','good','bad','source'])).reset_index(drop=True)
        # article_info = article_info.append(pd.DataFrame([[good, bad]], columns=['good', 'bad'])).reset_index(drop=True)

    except:
        print('실패')
        pass

# 형식 추가 (날짜 형식 수정 + 날짜별 내림차순 + 날짜와 순번을 결합한 IDX 추가)
article_info['NEWS_DATE'] = article_info['NEWS_DATE'].apply(lambda x: x.replace('오후','PM').replace('오전','AM'))
article_info['NEWS_DATE'] = article_info['NEWS_DATE'].apply(lambda x: datetime.datetime.strptime(x,"%Y.%m.%d. %p %I:%M"))

article_info = article_info.sort_values('NEWS_DATE',ascending=False).reset_index(drop=True)
article_info['IDX'] = [date.strftime('%y%m%d') + '{0:04d}'.format(i+1) for i,date in enumerate(article_info['NEWS_DATE'])]
article_info['clean_content'] = article_info['clean_content'].apply(lambda x: cleansing(x))
article_info['final_title'] = article_info['TITLE'].apply(lambda x: cleansing(x))
article_info['final_title'] = article_info['final_title'].apply(lambda x: extract_mecab_title(x))
article_info['final_content'] = article_info['clean_content'].apply(lambda x: extract_mecab(x))

article_info = article_info[['IDX','CATEGORY','NEWS_DATE','TITLE','clean_content','URL_DETAIL','final_title','final_content','good','bad','source']]


